In [ ]:

import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:

try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 1
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:

def append_path(pre):
    return np.vectorize(lambda file: os.path.join(GCS_DS_PATH,pre,file))

In [ ]:
sub = pd.read_csv('/kaggle/input/alaska2-image-steganalysis/sample_submission.csv')
train_filenames=np.array(os.listdir('/kaggle/input/alaska2-image-steganalysis/Cover/'))

In [ ]:
np.random.seed(0)
positives = train_filenames.copy()
negatives = train_filenames.copy()

np.random.shuffle(positives)
np.random.shuffle(negatives)

jmipod=append_path('JMiPOD')(positives[:10000])
juniward=append_path('JUNIWARD')(positives[10000:])
uerd=append_path('UERD')(positives[20000:30000])

pos_paths=np.concatenate([jmipod, juniward,uerd])

In [ ]:
test_paths=append_path('Test')(sub.Id.values)
neg_paths =append_path('Cover')(negatives[:30000])

In [ ]:
train_paths=np.concatenate([pos_paths,neg_paths])
train_labels= np.array([1] * len(pos_paths)+[0] * len(neg_paths))

In [ ]:
train_paths, valid_paths, train_labels, valid_labels=train_test_split(train_paths,train_labels,test_size=0.2,random_state=42)

In [ ]:
def decode_image(filename, label=None, image_size=(512,512)):
    bits=tf.io.read_file(filename)
    image=tf.image.decode_jpeg(bits,channels=3)
    image=tf.cast(image,tf.float32)#image to tf.float32 data type
    image=tf.image.resize(image,image_size)
    
    if label is None:
        return image
    else:
        return image,label

def data_augment(image,label=None):
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image,label

In [ ]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((train_paths,train_labels))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .cache()
                 .repeat()
                 .shuffle(1024)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )
valid_dataset= (tf.data.Dataset
                .from_tensor_slices((valid_paths,valid_labels))
                .map(decode_image, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
               )
test_dataset= (tf.data.Dataset
               .from_tensor_slices(test_paths)
               .map(decode_image, num_parallel_calls=AUTO)
               .batch(BATCH_SIZE)
              )

In [ ]:
def build_lrfn(lr_start=0.0001,lr_max=0.000075,
              lr_min=0.000001, lr_rampup_epochs=20,
              lr_sustain_epochs=0,lr_exp_decay=0.8):
    lr_max = lr_max * strategy.num_replicas_in_sync
    
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max- lr_start)/lr_rampup * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
            return lr
        
        return lrfn

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        ResNet50(
        input_shape=(512,512,3),
        weights='imagenet',
        include_top=False),
        L.GlobalAveragePooling2D(),
        L.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )
    model.summary()

In [ ]:
STEPS_PER_EPOCH=train_labels.shape[0] // BATCH_SIZE

history= model.fit(
train_dataset,
epochs=EPOCHS,
steps_per_epoch=STEPS_PER_EPOCH,
validation_data=valid_dataset)

In [ ]:
model.save('model.h5')